# MLlib RDD-Based

---

## Kiểm tra môi trường

In [1]:
import sys
sys.executable

'/usr/bin/python3'

## Import các thư viện cần thiết

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F, Row
from pyspark.mllib.tree import DecisionTree
from pyspark.mllib.evaluation import RegressionMetrics
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors
import pandas as pd

## Spark initialization

In [3]:
spark = SparkSession.builder \
    .appName("DecisionTreeRegressor") \
        .getOrCreate()
sc = spark.sparkContext

your 131072x1 screen size is bogus. expect trouble
25/05/02 09:22:18 WARN Utils: Your hostname, LAPTOP-I95CP2I9 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/05/02 09:22:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/02 09:22:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Load the dataset

- Dữ liệu từ các tệp sample_submission.csv, train.csv, test.csv được đọc bằng textFile, tạo ra các RDD, mỗi dòng là một chuỗi văn bản (string).
- Dữ liệu được lọc để loại bỏ dòng header khỏi mỗi RDD (tránh ảnh hưởng đến quá trình xử lý dữ liệu sau này).

In [212]:
import os
print(os.getcwd())
base_path = "/home/tienanh/22120017/Lab03_Spark/dataset/nyc-taxi-trip-duration"

/home/tienanh/22120017/Lab03_Spark/src/Regression/MLlib_RDD_Based


In [213]:
sample_data = sc.textFile(os.path.join(base_path, "sample_submission/sample_submission.csv"))
train_data = sc.textFile(os.path.join(base_path, "train/train.csv"))
test_data = sc.textFile(os.path.join(base_path, "test/test.csv"))

In [214]:
sample_data_header = sample_data.first()
train_data_header = train_data.first()
test_data_header = test_data.first()

sample_data = sample_data.filter(lambda line: line != sample_data_header)
train_data = train_data.filter(lambda line: line != train_data_header)
test_data = test_data.filter(lambda line: line != test_data_header)

In [215]:
# Show the first 5 rows 
for line in sample_data.take(5):
    print(line)

id3004672,959
id3505355,959
id1217141,959
id2150126,959
id1598245,959


In [216]:

for line in train_data.take(5):
    print(line)

id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982154846191406,40.767936706542969,-73.964630126953125,40.765602111816406,N,455
id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415344238281,40.738563537597656,-73.999481201171875,40.731151580810547,N,663
id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979026794433594,40.763938903808594,-74.005332946777344,40.710086822509766,N,2124
id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040283203125,40.719970703125,-74.01226806640625,40.706718444824219,N,429
id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973052978515625,40.793209075927734,-73.972923278808594,40.782520294189453,N,435


In [217]:

for line in test_data.take(5):
    print(line)

id3004672,1,2016-06-30 23:59:58,1,-73.988128662109375,40.732028961181641,-73.99017333984375,40.756679534912109,N
id3505355,1,2016-06-30 23:59:53,1,-73.964202880859375,40.67999267578125,-73.959808349609375,40.655403137207031,N
id1217141,1,2016-06-30 23:59:47,1,-73.9974365234375,40.737583160400391,-73.986160278320312,40.729522705078125,N
id2150126,2,2016-06-30 23:59:41,1,-73.956069946289063,40.771900177001953,-73.986427307128906,40.73046875,N
id1598245,1,2016-06-30 23:59:33,1,-73.97021484375,40.761474609375,-73.961509704589844,40.755889892578125,N


## Parsing the data 

- Chọn ra các cột dạng numeric là những đặc trưng cần thiết để phục vụ huấn luyện mô hình từ tập dữ liệu `train.csv`. Sau đó ép kiểu `float` 
- `train_data_parsed` thu được là một RDD của danh sách số thực, mỗi phần tử đại diện cho một chuyến đi với 5 đặc trưng + 1 nhãn.

In [ ]:

train_data_parsed = train_data.map(lambda line: line.split(",")).map(lambda parts: [
    float(parts[5]), float(parts[6]), #pickup_longitude, pickup_latitude
    float(parts[7]), float(parts[8]), #dropoff_longitude, dropoff_latitude
    float(parts[4]), #passenger_count
    float(parts[10]) #trip_duration
])

In [219]:
for line in train_data_parsed.take(5):
    print(line)

[-73.9821548461914, 40.76793670654297, -73.96463012695312, 40.765602111816406, 1.0, 455.0]
[-73.98041534423828, 40.738563537597656, -73.99948120117188, 40.73115158081055, 1.0, 663.0]
[-73.9790267944336, 40.763938903808594, -74.00533294677734, 40.710086822509766, 1.0, 2124.0]
[-74.01004028320312, 40.719970703125, -74.01226806640625, 40.70671844482422, 1.0, 429.0]
[-73.97305297851562, 40.793209075927734, -73.9729232788086, 40.78252029418945, 1.0, 435.0]


## Pre-process the data

-  Chia dữ liệu thành tập huấn luyện và kiểm tra:
   - Dữ liệu được chia thành hai phần: (training set) và (validation set).
   - Việc chia này đảm bảo rằng mô hình không "học thuộc" (overfitting) dữ liệu huấn luyện và có thể tổng quát tốt trên dữ liệu chưa thấy.
- Chuyển đổi thành `LabeledPoint`: đóng gói đặc trưng và nhãn thành cấu trúc `LabeledPoint` để phù hợp với các mô hình học máy trong Spark.

In [ ]:
train, validation = train_data_parsed.randomSplit([0.8, 0.2], seed=42)
rdd_train = train.map(lambda cols:
        LabeledPoint(cols[-1], Vectors.dense(cols[:-1]))
    )
rdd_test = validation.map(lambda cols:
        LabeledPoint(cols[-1], Vectors.dense(cols[:-1]))
    )

## Training

- Huấn luyện mô hình: Mô hình Decision Tree Regressor huấn luyện dự đoán các giá trị liên tục `trip_duration`
- Khởi tạo mô hình:
  - impurity='variance': Sử dụng độ lệch phương sai để phân chia các nhánh trong cây quyết định.
  - maxDepth=5: Giới hạn độ sâu của cây quyết định, giúp tránh việc mô hình quá phức tạp (overfitting).
  - maxBins=32: Giới hạn số lượng phân vùng có thể chia cho mỗi đặc trưng.

In [221]:
model_rdd_train = DecisionTree.trainRegressor(rdd_train, categoricalFeaturesInfo={},
                                              impurity='variance', maxDepth=5, maxBins=32)

### Evaluate the obtained model:

In [222]:
predictions = model_rdd_train.predict(rdd_test.map(lambda p: p.features))
labels_and_predictions = rdd_test.map(lambda p: p.label).zip(predictions)

In [223]:
labels_and_predictions.take(5)

[(1225.0, 777.5983592670409),
 (1128.0, 777.5983592670409),
 (211.0, 777.5983592670409),
 (251.0, 777.5983592670409),
 (652.0, 777.5983592670409)]

In [224]:
mse = labels_and_predictions.map(lambda pl: (pl[0] - pl[1])**2).mean() 
print("Mean Squared Error =", mse) 

Mean Squared Error = 10388783.835836709


**Nhận xét**
- MSE đo lường trung bình bình phương khoảng cách giữa giá trị dự đoán và thực tế. Ở đây, giá trị này hơn 10 triệu là khá lớn.
- Đơn vị của `trip_duration` là giây nên sai số bình phương trung bình khoảng: $$\sqrt{10388783.84} \approx 3223  second \approx 53.7  minute$$
- Điều này có nghĩa là mỗi lần dự đoán trung bình có thể lệch tới gần 54 phút so với thực tế.
- Cải thiện mô hình: 
  - Thêm đặc trưng mới như thời gian trong ngày, ngày trong tuần.v.v.
  - Tăng độ sâu của cây (maxDepth) để mô hình học kỹ hơn.
  - Chuẩn hóa dữ liệu nếu có outliner.
  - Thử mô hình khác như Random Forest hoặc Gradient Boosted Trees.

### Make prediction in test dataset

In [ ]:
test_data_parsed = test_data.map(lambda row: row.split(",")).map(
    lambda parts: (
        parts[0],  # unique trip ID
        [
            float(parts[4]), float(parts[5]),  # pickup_longitude, pickup_latitude
            float(parts[6]), float(parts[7]),  # dropoff_longitude, dropoff_latitude
            float(parts[3])  # passenger_count
        ]
    )
)

In [ ]:
test_data_local = test_data_parsed.collect()
predictions_local = [(row[0], model_rdd_train.predict(row[1])) for row in test_data_local]

In [ ]:
base_path_result = "/home/tienanh/22120017/Lab03_Spark/src/Regression/MLlib_RDD_Based"

In [ ]:
predictions_rdd = sc.parallelize(predictions_local)
predictions_df = predictions_rdd.map(lambda x: Row(id=x[0], trip_duration=x[1])).toDF()
predictions_df.coalesce(1).write.parquet(
    path=os.path.join(base_path, "result"),
    mode="overwrite"
)